## 영문 텍스트에 anki 단어장 만들기

### 실습 목표
- 영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성하는 프롬프트를 설계한다.
- 프롬프트 내에 시스템 지시사항, 예시, 출력 형식을 포함한다.

### 문제
다음 요구사항을 만족하는 LLM 프롬프트를 작성하시오.

###요구사항
1. 시스템 지시사항
- LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
- 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
- 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
- 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

2. 예시
- 입력 텍스트 예시:
```
The committee will convene next week to discuss the new policy proposals and reach a consensus.
```

- 출력 예시:
```
convene;to come together for a meeting;verb;The committee will convene next week.
consensus;general agreement;noun;The group reached a consensus after a long discussion.
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.
```

3. 출력 형식
- 각 카드: 단어 또는 ```어구;뜻;품사;영어 예문```
- 모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
- 불필요한 부가설명, 줄바꿈, 문장 없음

4. DataFrame 시각화
- LLM 응답을 파싱해서 다음과 같이 DataFrame으로 출력하세요.


In [ ]:
# 함수 리팩토링

def Anki(text, model='gpt-4o-mini', temperature=1, top_p=1):

  # 역할/페르소나/지시사항
  system_message = """
  영어 텍스트를 입력하면, 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성해 주세요.

  - LLM이 영어 텍스트에서 학습 가치가 있는 주요 단어 또는 어구를 추출하여 단어장으로 만들어야 한다.
  - 너무 쉬운 단어는 제외하고, 고유명사, 전문용어, 관용구 등도 포함한다.
  - 각 카드에는 (1)단어/어구, (2)뜻(영어 또는 한글), (3)품사, (4)예문(영어)을 포함한다.
  - 각 카드는 Anki에서 바로 쓸 수 있는 포맷(각 항목은 세미콜론 ; 으로 구분, 한 줄에 한 카드)으로 출력한다.

  # Output Format
  - 각 카드 포함 항목 : 단어 또는 어구;뜻(영어 또는 한글);품사;영어 예문
  - 모든 카드는 한 줄에 한 개, 세미콜론(;)으로 필드 구분
  - 불필요한 부가설명, 줄바꿈, 문장 없음

  # Example
  - 입력: The committee will convene next week to discuss the new policy proposals and reach a consensus.
  - 출력:
    - convene;to come together for a meeting;verb;The committee will convene next week.
    - consensus;general agreement;noun;The group reached a consensus after a long discussion.
    - proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

  """

  user_message = f"""
  다음 영어 텍스트에서 중요한 어휘를 추출해 Anki 카드 형식의 단어장 데이터를 생성해 주세요.

  {text}
  """

  response = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "system",
        "content": [
          {
            "type": "text",
            "text": system_message
          }
        ]
      },
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": user_message
          }
        ]
      }
    ],
    response_format={
      "type": "text"
    },
    temperature=temperature,
    max_completion_tokens=2048,
    top_p=top_p,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response.choices[0].message.content

text = 'Despite initial skepticism, the groundbreaking research was eventually recognized as a pivotal contribution to the field of neuroscience.'
output = Anki(text)
print(output)

groundbreaking;innovative or unprecedented;adjective;The groundbreaking research changed the way we understand the brain.  
skepticism;doubt as to the truth of something;noun;Her skepticism about the results led to further testing.  
pivotal;of crucial importance in relation to the development or success of something;adjective;This discovery was pivotal for advancements in medical science.  
contribution;the act of giving or doing something;noun;His contribution to the project was invaluable.  
neuroscience;the scientific study of the nervous system;noun;Neuroscience has made significant progress in understanding brain disorders.


In [ ]:
import pandas as pd

# 각 줄을 파싱하여 리스트로 변환
cards = [line.strip().split(";") for line in output.strip().split("\n")]

# DataFrame 생성
df = pd.DataFrame(cards, columns=["Word/Phrase", "Definition", "PoS", "Example Sentence"])

# 출력
df

,Word/Phrase,Definition,PoS,Example Sentence
0,groundbreaking,innovative or unprecedented,adjective,The groundbreaking research changed the way we...
1,skepticism,doubt as to the truth of something,noun,Her skepticism about the results led to furthe...
2,pivotal,of crucial importance in relation to the devel...,adjective,This discovery was pivotal for advancements in...
3,contribution,the act of giving or doing something,noun,His contribution to the project was invaluable.
4,neuroscience,the scientific study of the nervous system,noun,Neuroscience has made significant progress in ...
